##transformers의 모델 클래스 불러오기
transformers 라이브러리에서는 각종 태스크에 맞게 BERT 위에 출력층을 추가한 모델 클래스 구현체를
제공하고 있습니다. 아래의 구현체를 사용하면 사용자가 별도의 출력층을 설계할 필요없이 태스크에 맞
게 모델을 로드하여 사용할 수 있습니다. 이 책에서는 모델 구조의 이해를 위해 모든 실습에서 출력층을
직접 설계하여 실습을 진행하지만, 이미 모델의 구조를 이해한 상황에서는 아래와 같이 이미 출력층이 설
계된 모델들을 사용하는 것이 훨씬 코드 작성이 간편합니다.


###1. 다 대 일 유형
다 대 일 (many‐to‐one) 유형은 텍스트 분류에 주로 사용되는 모델입니다.  TFBertForSequence‐
Classification’ 실습에서는 아래의 모델을 사용해봅니다.

In [1]:
# from transformers import TFBertForSequenceClassification
# model = TFBertForSequenceClassification.from_pretrained("모델 이름", num_labels = 분류한 레이블의 개수)

### 2. 다 대 다 유형
다 대 다 (many‐to‐one) 유형은 다수의 입력에 대해서 다수의 출력이 필요할 때 사용하는 모델입니다.
'개체명 인식’ 이 해당 모델을 사용하는 대표적인 예시입니다.

In [2]:
# from transformers import TFBertForTokenClassification
# model = TFBertForTokenizerClassification.from_pretrained("모델 이름", num_labels=분류할 레이블의 개수)

###3. 질의 응답 유형
질의 응답 (Question Answering) 문제를 풀기 위해 사용하는 모델로 ‘기계 독해’ 실습이 해당 모델을
사용할 수 있는 대표적인 예시입니다.

In [3]:
# from transformers import TFBertForQuestionAnswering
# model = TFBertForQuestionAnswering.from_pretrained('모델 이름')

## 한국어 BERT를 이용한 네이버 영화 리뷰 분류


In [4]:
!pip install transformers

### 1. 데이터 로드 및 정제


In [5]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f9c9a40e790>)

In [7]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [8]:
print('훈련용 리뷰의 개수 : ', len(train_data))
print('테스트용 리뷰의 개수 : ', len(test_data))

훈련용 리뷰의 개수 :  150000
테스트용 리뷰의 개수 :  50000


훈련 데이터와 테스트 데이터의 리뷰 개수는 각각 15 만개와 5 만개입니다. 중복 데이터와 결측값을 제거
합니다.

In [9]:
# document 열에서 중복인 내용이 있다면 중복제거
train_data.drop_duplicates(subset=['document'], inplace=True)

train_data = train_data.dropna(how='any') # 널값이 존재하는 행 제거
print('훈련 데이터의 리뷰 수 : ', len(train_data))

훈련 데이터의 리뷰 수 :  146182


In [10]:
test_data = test_data.dropna(how = 'any')
print('테스트 데이터의 리뷰 수 :',len(test_data))

테스트 데이터의 리뷰 수 : 49997


###2. BERT의 입력
사전 학습된 BERT 류의 모델들의 경우에는 BertTokenizer.from_pretrained(‘모델 이름’) 을 넣으면 해당
모델이 학습되었을 당시에 사용되었던 토크나이저를 로드합니다. klue/bert‐base 의 토크나이저를 로드
해봅시다.

In [11]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [12]:
print(tokenizer.tokenize('보는내내 그대로 들어맞는 예측 카리스마 없는 악역'))

['보', '##는', '##내', '##내', '그대로', '들어맞', '##는', '예측', '카리스마', '없', '##는', '악역']


토크나이저의 encode()를 사용하면 정수 인코딩 결과를 얻을 수 있습니다.

In [13]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[2, 1160, 2259, 2369, 2369, 4311, 20657, 2259, 5501, 13132, 1415, 2259, 23713, 3]


여기서 주의할 점은 앞의 2 번과 뒤에 붙은 3 번은 원래 있던 단어가 아니라는 점입니다. decode() 를 사용
하면 정수 인코딩 결과를 다시 텍스트로 변환합니다.

In [14]:
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

'[CLS] 보는내내 그대로 들어맞는 예측 카리스마 없는 악역 [SEP]'

맨 앞에 [CLS] 와 [SEP] 토큰이 출력되는데 이 토큰들은 각각 encode() 를 호출하였을 때, 정수 인코딩 과
정에서 자동으로 앞, 뒤에 부착한 2 번과 3 번에 해당됩니다. 토크나이저는 특별 토큰에 대해서는 바로 맵
핑된 정수값을 확인할 수 있도록 지원하고 있습니다. 실제로 2 번과 3 번이 [CLS] 토큰과 [SEP] 토큰이 맞
는지 확인해봅시다.

In [15]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':', tokenizer.sep_token_id)

[CLS] : 2
[SEP] : 3


In [16]:
# 패딩 토큰도 확인해보겠습니다.
print(tokenizer.pad_token, ':', tokenizer.pad_token_id)

[PAD] : 0


패딩 토큰은 0 번임을 확인했습니다. encode() 를 할 때, 최대 길이를 지정하고 해당 길이까지 패딩하는 것
도 가능합니다. max_length 의 값으로 최대 길이를 지정해주고 pad_to_max_length 의 값을 True 로 하여 정수 인코딩을 하는 동시에 최대 길이까지 패딩해봅시다.

In [17]:
max_seq_len = 144

# pad_to_max_length = True는 transformers라이브러리에서 더이상 지원하지 않음 대신 padding = 'max_length'
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화",
                                  max_length=max_seq_len, padding = 'max_length')

print(encoded_result)
print('길 이 :', len(encoded_result))

[2, 1537, 2534, 2069, 6572, 2259, 3771, 18, 3690, 4530, 2585, 2073, 3771, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길 이 : 144


위와 같은 정수 인코딩 결과 외에도 BERT 가 필요로 하는 입력이 두 가지가 더 있습니다. 하나는 문장 구
분을 위한 세그먼트 인코딩, 나머지 하나는 실제 단어 토큰과 패딩 토큰을 구분하기 위한 어텐션 마스크입
니다. 우선 세그먼트 인코딩을 만들어봅시다. 영화 리뷰 분류는 문장이 구분되는 문제가 아닙니다. 이 경
우 한 종류의 문장만 있으므로 입력의 길이만큼 0 의 시퀀스를 만듭니다.

In [18]:
# 세그먼트 인코딩
print([0]*max_seq_len)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [19]:
# 어텐션 마스크 인코딩
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))
print(valid_num * [1] + (max_seq_len - valid_num)*[0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


입력된 전체 데이터에 대해서 이 과정을 진행하는 함수를 만듭니다.

In [20]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

  for example, label in tqdm(zip(examples, labels), total=len(examples)):
    # input_id는 워 드 임 베 딩 을 위 한 문 장 의 정 수 인 코 딩
    input_id = tokenizer.encode(example, max_length=max_seq_len, padding = 'max_length')

    # attention_mask는 실 제 단 어 가 위 치 하 면 1, 패 딩 의 위 치 에 는 0인 시 퀀 스.
    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

    # token_type_id은 세 그 먼 트 인 코 딩
    token_type_id = [0] * max_seq_len

    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {}vs {}".format(len(token_type_id), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  data_labels = np.asarray(data_labels, dtype=np.int32)


  return (input_ids, attention_masks, token_type_ids), data_labels

훈련 데이터에 대해서 진행합니다.

In [21]:
train_X, train_y = convert_examples_to_features(train_data['document'], train_data['label'], max_seq_len = max_seq_len, tokenizer = tokenizer)

100%|██████████| 146182/146182 [00:46<00:00, 3127.26it/s]


테스트 데이터에 대해서 진행합니다.

In [22]:
test_X, test_y = convert_examples_to_features(test_data['document'], test_data['label'], max_seq_len = max_seq_len, tokenizer = tokenizer)

100%|██████████| 49997/49997 [00:15<00:00, 3280.19it/s]


훈련 데이터의 첫번째 샘플에 대해서 출력해보겠습니다.

In [23]:
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label =train_y[0]

print('단 어 에 대 한 정 수 인 코 딩 :',input_id)
print('어 텐 션 마 스 크 :',attention_mask)
print('세 그 먼 트 인 코 딩 :',token_type_id)
print('각 인 코 딩 의 길 이 :', len(input_id))
print('정 수 인 코 딩 복 원 :',tokenizer.decode(input_id))
print('레 이 블 :',label)

단 어 에 대 한 정 수 인 코 딩 : [   2 1376  831 2604   18   18 4229 9801 2075 2203 2182 4243    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
어 텐 션 마 스 크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 

### BERT의 출력 이해하기
BERT 를 이용해 모델을 구현하기 위해서는 BERT 의 출력을 이해할 필요가 있습니다. 우선, 한국어 BERT
인 klue/bert‐base 를 로드해봅시다.

In [24]:
model = TFBertModel.from_pretrained("klue/bert-base", from_pt = True)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClas

BERT 의 출력을 outpus 이라는 변수에 저장합니다. 입력 문장의 길이는 144 로 가정합니다.

In [25]:
max_seq_len = 144
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype = tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype = tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype = tf.int32)
outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

In [26]:
# 문장의 길이 만큼 출력
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 144, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model'")


In [27]:
# CLS 토큰의 위치 출력
print(outputs[1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/pooler/dense/Tanh:0', description="created by layer 'tf_bert_model'")


outputs[0] 은 (배치 크기, 144, 768) 의 크기를 가지는 텐서입니다. 이는 768 차원의 벡터가 144 개가 있다
는 의미로 문장 길이 개수만큼의 출력을 얻었음을 의미합니다. Many‐to‐Many 문제를 풀 때는 outputs[0]
을 사용하면 됩니다. 예시로는 개체명 인식이 있습니다. 반면, outputs[1] 은 (배치 크기, 768) 의 크기를
가지는 텐서입니다. 이는 [CLS] 토큰 위치의 출력으로 Many‐To‐One 문제를 풀 때는 outputs[1] 을 사용하
면 됩니다. 예시로는 텍스트 분류 문제로 지금 푸는 네이버 영화 리뷰 문제가 이에 해당됩니다.

###4. BERT를 이용한 Many-to-One모델 만들기
서브클래싱 구현 방식으로 구현한 Many‐to‐One 모델은 다음과 같습니다. BERT 의 출력 중 outputs[1].
즉, [CLS] 토큰 위치의 출력을 1 개의 뉴런을 가지며 시그모이드 함수가 활성화 함수로 설정된 출력층으로
연결합니다.

In [28]:
class TFBertForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name):
    super(TFBertForSequenceClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt = True)
    self.classifier = tf.keras.layers.Dense(1, kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02), activation='sigmoid', name='classifier')

  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
    cls_token = outputs[1]
    prediction = self.classifier(cls_token)

    return prediction


배치 크기는 64로 하고 훈련 데이터의 20%를 검증 데이터로 사용하여 2 에포크 학습합니다.

In [29]:
model = TFBertForSequenceClassification("klue/bert-base")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [30]:
model.fit(train_X, train_y, epochs=2, batch_size=64, validation_split=0.2)

Epoch 1/2
1828/1828 [==============================] - 428s 212ms/step - loss: 0.2835 - accuracy: 0.8803 - val_loss: 0.2367 - val_accuracy: 0.9031
Epoch 2/2
1828/1828 [==============================] - 379s 207ms/step - loss: 0.1866 - accuracy: 0.9264 - val_loss: 0.2765 - val_accuracy: 0.8945


테스트 데이터에 대해서 로스와 정확도를 계산합니다.

In [31]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc", results)

49/49 [==============================] - 41s 843ms/step - loss: 0.2844 - accuracy: 0.8887
test loss, test acc [0.2844456434249878, 0.8886733055114746]


### 5. 리뷰 예측 해보기


In [32]:
def sentiment_predict(new_sentence):
  input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, padding='max_length')

  padding_count = input_id.count(tokenizer.pad_token_id)
  attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
  token_type_id = [0] * max_seq_len

  input_ids = np.array([input_id])
  attention_masks = np.array([attention_mask])
  token_type_ids = np.array([token_type_id])

  encoded_input = [input_ids, attention_masks, token_type_ids]
  score = model.predict(encoded_input)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score*100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1-score) *100))

In [33]:
sentiment_predict("이 영 화 존 잼 입 니 다 대 박")

1/1 [==============================] - 4s 4s/step
93.31% 확률로 긍정 리뷰입니다.



In [34]:
sentiment_predict('이 영 화 개 꿀 잼 ᄏ ᄏ ᄏ')

1/1 [==============================] - 0s 41ms/step
98.68% 확률로 긍정 리뷰입니다.



In [35]:
sentiment_predict('이 영 화 핵 노 잼 ᅲ ᅲ')

1/1 [==============================] - 0s 39ms/step
94.16% 확률로 부정 리뷰입니다.



In [36]:
sentiment_predict('이 딴 게 영 화 냐 ᄍ ᄍ')

1/1 [==============================] - 0s 38ms/step
84.02% 확률로 부정 리뷰입니다.



In [37]:
sentiment_predict('감 독 뭐 하 는 놈 이 냐?')

1/1 [==============================] - 0s 39ms/step
72.07% 확률로 긍정 리뷰입니다.



In [38]:
sentiment_predict('와 개 쩐 다 정 말 세 계 관 최 강 자 들 의 영 화 다')

1/1 [==============================] - 0s 39ms/step
75.47% 확률로 긍정 리뷰입니다.

